In [ ]:
import os
import numpy as np
from flask import Flask
import cv2
from tensorflow.keras.models import load_model

IM_SIZE = 33

app = Flask(__name__)
cap = cv2.VideoCapture(0)
model = load_model('good_model.h5')

@app.route('/video_feed')
def predict_video():
    global cap
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    labels_dict={1:'with_mask',0:'without_mask'}
    color_dict={0:(0,0,255),1:(0,255,0)}

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            face_img=gray[y:y+w,x:x+w]        
            resized=cv2.resize(face_img,(IM_SIZE,IM_SIZE))
            normalized=resized/255.0
            reshaped=np.reshape(normalized,(1,IM_SIZE,IM_SIZE,1))
            result=model.predict(reshaped)
            label=np.argmax(result,axis=1)[0]
            cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
            cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
            cv2.putText(frame, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)

        cv2.imshow('LIVE',frame)
        key=cv2.waitKey(1)

        if(key==27):
            break

    cv2.destroyAllWindows()
    cap.release()

if __name__ == '__main__':
    port = os.environ.get('PORT')

    if port:
        app.run(host='0.0.0.0', port=int(port))
    else:
        app.run()